In [2]:
import pandas as pd
from camel_tools.tokenizers.word import simple_word_tokenize
from tashaphyne.stemming import ArabicLightStemmer
from sklearn import preprocessing
import numpy as np

In [ ]:
# pip install Tashaphyne

In [2]:
ArListem = ArabicLightStemmer()

In [3]:
df = pd.read_csv(r'CSVs/foNvdia.csv' ,encoding='utf-8')
arabicLatter = pd.read_csv(r'CSVs/arabiclatter.csv' ,encoding='utf8')
arabicLatter =arabicLatter.iloc[:,1:]
arabicLatter.describe()
import ast
res = []
for i in range(len(df)):
    res.append(ast.literal_eval(df['CodaCODE'].iloc[i]))
df['CodaCODE20'] = res
df

,English,MSA,Dialect,CODA,SentimentValue,species,CodaCODE,CodaCODE20
0,number,رَقْم ، عَدَد,AMM,رقم,2,441,"[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,number,رَقْم ، عَدَد,JER,رقم,2,441,"[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,number,رَقْم ، عَدَد,BEI,رقم,2,441,"[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,number,رَقْم ، عَدَد,AMM,رقم,2,441,"[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,number,رَقْم ، عَدَد,ASW,رقم,2,441,"[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 21, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
46462,Car traffic,زحام,NaN,زحمة,0,457,"[11, 6, 24, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 6, 24, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
46463,Car traffic,زحام,NaN,زحمه,0,457,"[11, 6, 24, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 6, 24, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
46464,garbage,زبالة,NaN,زبالة,0,456,"[11, 2, 1, 23, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 2, 1, 23, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
46465,garbage,زبالة,NaN,زباله,0,456,"[11, 2, 1, 23, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 2, 1, 23, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
# load json and create model
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('LstmModel/withoutStem73.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("LstmModel/withoutStem73.h5")
print("Loaded model from disk")

Loaded model from disk


In [8]:
df1 = pd.read_csv(r'CSVs/validationDataset_1K_reviews.csv',encoding = 'utf-8')
df1

,Unnamed: 0,rating,review,SentimantValue,predectedValue
0,0,5,“ممتاز”. كل شي. لا يوجد,1,1
1,1,5,“ممتاز جدا”. نظيف .. وراقي جدا ...,1,1
2,2,5,استثنائي. فندق نظيف وممتاز وحسن استفبال وخذمه ...,1,1
3,3,4,“أستمتع فى أجواء الكرنيش”. الموقع و راحه وخدما...,1,1
4,4,5,استثنائي. التعامل اللطيففخامة الغرفة و ملحقاته...,1,1
...,...,...,...,...,...
995,995,2,ضعيف سعة الشقه رائحة الشقه والاستقبال والبعد ع...,0,1
996,996,2,مخيب للأمل لم يعجبني عند دخولي إلى الفندق تبي...,0,1
997,997,1,ضعيف جداً لا شي مادي بحت,0,1
998,998,2,ضعيف لا تستطيع النوم من اصوت الموسيقى,0,1


In [9]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(np.array(df1['SentimantValue'].tolist()),np.array(df1['predectedValue'])))
print(classification_report(np.array(df1['SentimantValue'].tolist()),np.array(df1['predectedValue'])))

[[285  13   2]
 [  9 684   7]
 [  0   0   0]]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       300
           1       0.98      0.98      0.98       700
           2       0.00      0.00      0.00         0

    accuracy                           0.97      1000
   macro avg       0.65      0.64      0.65      1000
weighted avg       0.98      0.97      0.97      1000



F:\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# The Algorithm that extract the "predectedValue" on the dataframe 

In [ ]:
pridRes = []
for tweet in range (len(df1)):
    lioo = [] 
    listDf = []
    sentence = df1.iloc[tweet]['review']
    sentence = sentence.replace('“', '').replace('،', '').replace(',',"").replace('ء',"").replace('؛','').replace('"','').replace('?','').replace('كل','').replace('شئ','').replace(' عن ',' ').replace(' او ',' ').replace('اي',' ').replace(' من ',' ').replace('ولاشي',' ').replace('شي',' ').replace('  ',' ').replace('موظف', '')
    sentence = simple_word_tokenize(sentence)
    for i in range(len(sentence)):  
            ddf = sentence[i]
            if ((sentence[i] == 'جدا') or (sentence[i] == 'جداً')):
                if(i >0):
                    print("here ", i)
                    listDf.append(listDf[-1])
            else:
                wordToNorm =  ArListem.normalize(ddf)
                wordToNorm =   wordToNorm[0:3].replace("وال","")+wordToNorm[3:] 
                wordToNorm =   wordToNorm[0:3].replace("بال","")+wordToNorm[3:] 
                wordToNorm =   wordToNorm[0:2].replace("ال","")+wordToNorm[2:]
                wordToNorm =   wordToNorm[0:2].replace("وب","")+wordToNorm[2:]
                wordToNorm =   wordToNorm[0:2].replace("لل","")+wordToNorm[2:]
                wordToNorm =  wordToNorm[:-2] + wordToNorm[-2:].replace("ها","")
                wordToNorm =  wordToNorm[:-1] + wordToNorm[-1:].replace("ه","")
                wordToNorm1 = ''
                for p in range(len(wordToNorm)):
                    if (p)==0:
                        wordToNorm1 += wordToNorm[p]
                        pass
                    else:
                        if wordToNorm[p - 1]  == wordToNorm[p]:
                            pass
                        else:
                            wordToNorm1 +=  wordToNorm[p]
                nListTest = []
                for x in wordToNorm1 :
                    tr = 0
                    if (x == '+') or (x == '#') or ( x == '“'):
                        print('+#')
                    else:
                        if x  == ' ':
                            tr = 33
                        else:
                            for i in range(0,len(arabicLatter)-1):
                                if (arabicLatter.iloc[i]['chars'] == x ) :
                                    tr = i+1
                    nListTest.append(tr) 
                nListTest
                tList =  list(filter(lambda num: num != 0, nListTest))
                b = [0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0,0,0,0,0, 0, 0, 0, 0]
                nListTest.extend(b[len(tList):])
                dd = np.array([nListTest])
                dd
                scores = loaded_model.predict(dd)
                if (df[df['species'] == np.argmax([scores]) ][['MSA','SentimentValue','CODA']].drop_duplicates('MSA')['SentimentValue'].tolist()[0]) == 0:
                     listDf.append(0)
                elif df[df['species'] == np.argmax([scores]) ][['MSA','SentimentValue','CODA']].drop_duplicates('MSA')['SentimentValue'].tolist()[0] == 1:
                     listDf.append(1)
                else:
                     listDf.append(2)
    if listDf.count(0) > listDf.count(1):
        pridRes.append(0)
    elif listDf.count(0) == listDf.count(1) :
        pridRes.append(2)
    else:
        pridRes.append(1)
pridRes            
            